In [13]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cross_validation import train_test_split


filepath = 'PartD_Prescriber_PUF_NPI_Drug_13.txt'
# minimum number of samples per class
mincounts = 5
# number of rows of csv file to import
numrows = 100000
# train:test ratio for splitting the dataset
ratio = 0.3

In [14]:
# pre-processing

data = pd.read_csv(filepath, delimiter="\t", nrows = numrows)

newdf = data[['nppes_provider_last_org_name', 'specialty_description', 'drug_name']]
# drop nan rows
newdf = newdf.dropna(axis=1, how='any')
# drop classes with number of samples less than predefined mincounts - not enough data
g = newdf.groupby(by = 'specialty_description').filter(lambda x: len(x) > mincounts)
# newdf.groupby('specialty_description').size()
features = g.as_matrix(columns = ['nppes_provider_last_org_name','drug_name'])
labels = g.as_matrix(columns = ['specialty_description'])

# concatenate features into one vectors
X = []
for i in range(len(features)):
    temp = []
    for word in features[i]:
        temp.extend(word.split())
    X.append(temp)

In [15]:
# since our feature matrix is sparse - use scipy.csr sparse matrix 
indices = []
data = []
indptr = [0]
vocab = {}
for row in X:
    for term in row:
        index = vocab.setdefault(term, len(vocab))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

X = csr_matrix((data, indices, indptr), dtype = int).toarray()
print 'Finished constructing CSR matrix for features... \nIt is a matrix of {}.'.format(X.shape)

Finished constructing CSR matrix for features... 
It is a matrix of (99958, 4535).


In [16]:
# get ground truth labels

_, idx = np.unique(labels, return_index=True)
gtlabels = labels[np.sort(idx)]


In [17]:
# splitting data into train and test
labels = np.reshape(labels, (len(labels), ))
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = ratio)
train_samples, n_features = x_train.shape
n_classes = len(gtlabels)

print train_samples, n_features
print n_classes

69970 4535
66


In [7]:
from sklearn.linear_model import LogisticRegression
solver = 'saga'

lr = LogisticRegression(solver=solver, 
                        multi_class= 'multinomial', 
                        C=1,
                        penalty = 'l1',
                        fit_intercept=True,
                        max_iter=10,
                        random_state=42, 
                       )
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)
np.sum(y_pred == y_test) *1./ y_test.shape[0]

# receiving convergence warning on data of 100000 - possible solutions are : 1. increase max iteraction (right now
# it is 10) or 2. data simply can't be fitted by a logistic model

/Users/hujiayu/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.91593303988261976

In [8]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)
np.mean(predicted == y_test)    

0.86051087101507273

In [18]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',
              alpha=1e-3, random_state=42,
              max_iter=10, tol=None).fit(x_train, y_train)
predicted = clf.predict(x_test)
np.mean(predicted == y_test)    

0.8666133119914633

In [10]:
# take a look at precision vs. recall metrics

from sklearn import metrics

print(metrics.classification_report(y_test, predicted,
                                    gtlabels))

                                                                precision    recall  f1-score   support

                                             Internal Medicine       0.92      0.93      0.93      7559
                                                Anesthesiology       0.86      0.80      0.83        88
                                                       Dentist       0.81      0.59      0.68       223
                                            Nurse Practitioner       0.93      0.82      0.87      3106
                                               Family Practice       0.79      0.97      0.87      9594
                                         Obstetrics/Gynecology       0.85      0.52      0.65       258
                                           Physician Assistant       0.95      0.75      0.84      1637
                                                    Cardiology       0.94      0.84      0.89      1012
                                                   Dermatology 

/Users/hujiayu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# check for overfitting - see there is a large gap between train accuracy and test accuracy
predicted = clf.predict(x_train)
np.mean(predicted == y_train)  

0.88183507217378876

In [12]:
# unique features vs. unique labels
# to see how many labels sort of share the same features 
# if # of unique features > # of unique labels - some classes share exactly the same features
# which will cause difficulty in learning to discriminate
print np.unique(features).shape[0]
print np.unique(labels).shape[0]

4464
66
